# Apache Airflow Deep Dive

Apache Airflow is an open-source platform for developing, scheduling, and monitoring batch-oriented workflows. Originally created at Airbnb, it has become the de facto standard for orchestrating complex data pipelines.

---

## Table of Contents
1. [Airflow Architecture](#architecture)
2. [DAG Structure](#dag-structure)
3. [Operators](#operators)
4. [XCom for Task Communication](#xcom)
5. [Best Practices for DAG Design](#best-practices)
6. [DAG Examples](#examples)
7. [Takeaways](#takeaways)

---

## 1. Airflow Architecture <a id='architecture'></a>

Airflow follows a **distributed architecture** with several key components working together:

```
┌─────────────────────────────────────────────────────────────────────────┐
│                          AIRFLOW ARCHITECTURE                           │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                         │
│    ┌──────────────┐     ┌──────────────┐     ┌──────────────┐          │
│    │   Web UI     │     │  Scheduler   │     │   Executor   │          │
│    │  (Flask)     │     │              │     │              │          │
│    └──────┬───────┘     └──────┬───────┘     └──────┬───────┘          │
│           │                    │                    │                  │
│           │                    ▼                    │                  │
│           │           ┌──────────────┐              │                  │
│           └──────────►│   Metadata   │◄─────────────┘                  │
│                       │   Database   │                                 │
│                       │  (Postgres)  │                                 │
│                       └──────────────┘                                 │
│                              ▲                                         │
│                              │                                         │
│    ┌─────────────────────────┴─────────────────────────┐               │
│    │                                                   │               │
│    ▼                    ▼                    ▼         │               │
│ ┌────────┐         ┌────────┐         ┌────────┐       │               │
│ │Worker 1│         │Worker 2│         │Worker N│       │               │
│ └────────┘         └────────┘         └────────┘       │               │
│                                                                         │
└─────────────────────────────────────────────────────────────────────────┘
```

### Core Components

| Component | Description | Responsibilities |
|-----------|-------------|------------------|
| **Scheduler** | The brain of Airflow | Parses DAGs, monitors task states, triggers task instances, manages dependencies |
| **Webserver** | Flask-based UI | Provides DAG visualization, task logs, trigger/pause DAGs, user authentication |
| **Workers** | Task executors | Execute the actual task logic, report status back to scheduler |
| **Metadata DB** | State storage (PostgreSQL/MySQL) | Stores DAG runs, task instances, variables, connections, XComs |
| **Executor** | Task distribution strategy | Determines how tasks are run (Local, Celery, Kubernetes, etc.) |

### Executor Types

| Executor | Use Case | Scalability |
|----------|----------|-------------|
| `SequentialExecutor` | Development/testing | Single task at a time |
| `LocalExecutor` | Small to medium workloads | Parallel on single machine |
| `CeleryExecutor` | Production, distributed | Horizontal scaling with message broker |
| `KubernetesExecutor` | Cloud-native, dynamic | Each task runs in its own pod |

---

## 2. DAG Structure <a id='dag-structure'></a>

A **DAG (Directed Acyclic Graph)** is a collection of tasks with defined dependencies and execution order.

### Key DAG Concepts

```
DAG Lifecycle:

┌─────────────┐    ┌─────────────┐    ┌─────────────┐    ┌─────────────┐
│   Parsed    │───►│  Scheduled  │───►│   Queued    │───►│   Running   │
└─────────────┘    └─────────────┘    └─────────────┘    └─────────────┘
                                                                │
                   ┌─────────────┐    ┌─────────────┐          │
                   │   Failed    │◄───│   Success   │◄─────────┘
                   └─────────────┘    └─────────────┘
```

### DAG Parameters

| Parameter | Description | Example |
|-----------|-------------|--------|
| `dag_id` | Unique identifier | `"my_etl_pipeline"` |
| `schedule_interval` | How often DAG runs | `"@daily"`, `"0 0 * * *"` |
| `start_date` | When DAG becomes active | `datetime(2024, 1, 1)` |
| `catchup` | Backfill past runs | `True` / `False` |
| `default_args` | Default task arguments | `{"retries": 3}` |
| `tags` | DAG categorization | `["etl", "production"]` |
| `max_active_runs` | Concurrent DAG runs | `1`, `3`, etc. |

In [ ]:
# Basic DAG Structure Example
from datetime import datetime, timedelta
from airflow import DAG
from airflow.operators.empty import EmptyOperator

# Default arguments applied to all tasks
default_args = {
    'owner': 'data_team',
    'depends_on_past': False,
    'email': ['alerts@company.com'],
    'email_on_failure': True,
    'email_on_retry': False,
    'retries': 3,
    'retry_delay': timedelta(minutes=5),
    'execution_timeout': timedelta(hours=2),
}

# DAG Definition
with DAG(
    dag_id='example_basic_dag',
    default_args=default_args,
    description='A simple example DAG',
    schedule_interval='@daily',  # or cron: '0 0 * * *'
    start_date=datetime(2024, 1, 1),
    catchup=False,
    tags=['example', 'tutorial'],
    max_active_runs=1,
) as dag:
    
    start = EmptyOperator(task_id='start')
    end = EmptyOperator(task_id='end')
    
    # Define task dependencies
    start >> end

---

## 3. Operators <a id='operators'></a>

Operators define **what** a task does. They are the building blocks of workflows.

### Operator Categories

```
┌────────────────────────────────────────────────────────────────┐
│                      OPERATOR TYPES                           │
├────────────────────┬────────────────────┬─────────────────────┤
│   Action Operators │  Transfer Operators│   Sensor Operators  │
├────────────────────┼────────────────────┼─────────────────────┤
│ • PythonOperator   │ • S3ToRedshiftOp   │ • FileSensor        │
│ • BashOperator     │ • GCSToGCSOperator │ • S3KeySensor       │
│ • EmailOperator    │ • MySqlToHiveOp    │ • HttpSensor        │
│ • DockerOperator   │ • PostgresToS3Op   │ • ExternalTaskSensor│
│ • KubernetesPodOp  │                    │ • SqlSensor         │
└────────────────────┴────────────────────┴─────────────────────┘
```

In [ ]:
# PythonOperator - Execute Python Callables
from airflow.operators.python import PythonOperator
from airflow import DAG
from datetime import datetime

def extract_data(**context):
    """Extract data from source."""
    print(f"Extracting data for {context['ds']}")
    data = {'records': 1000, 'source': 'api'}
    return data  # This will be pushed to XCom

def transform_data(ti, **context):
    """Transform extracted data."""
    # Pull data from previous task
    data = ti.xcom_pull(task_ids='extract')
    print(f"Transforming {data['records']} records")
    data['transformed'] = True
    return data

def load_data(ti, **context):
    """Load data to destination."""
    data = ti.xcom_pull(task_ids='transform')
    print(f"Loading {data['records']} transformed records")

with DAG(
    'python_operator_example',
    start_date=datetime(2024, 1, 1),
    schedule_interval='@daily',
    catchup=False,
) as dag:
    
    extract = PythonOperator(
        task_id='extract',
        python_callable=extract_data,
        provide_context=True,  # Deprecated in 2.0+, context is auto-provided
    )
    
    transform = PythonOperator(
        task_id='transform',
        python_callable=transform_data,
    )
    
    load = PythonOperator(
        task_id='load',
        python_callable=load_data,
    )
    
    extract >> transform >> load

In [ ]:
# BashOperator - Execute Shell Commands
from airflow.operators.bash import BashOperator
from airflow import DAG
from datetime import datetime

with DAG(
    'bash_operator_example',
    start_date=datetime(2024, 1, 1),
    schedule_interval='@daily',
    catchup=False,
) as dag:
    
    # Simple command
    check_date = BashOperator(
        task_id='check_date',
        bash_command='date',
    )
    
    # Command with templating
    process_file = BashOperator(
        task_id='process_file',
        bash_command='echo "Processing data for {{ ds }}"',
    )
    
    # Run external script
    run_script = BashOperator(
        task_id='run_script',
        bash_command='/opt/scripts/etl.sh {{ ds }} ',  # Note trailing space!
        env={'DATA_PATH': '/data/raw'},
    )
    
    # Chain with environment variables
    cleanup = BashOperator(
        task_id='cleanup',
        bash_command='rm -rf /tmp/airflow_temp_{{ ds_nodash }}',
        trigger_rule='all_done',  # Run even if upstream fails
    )
    
    check_date >> process_file >> run_script >> cleanup

In [ ]:
# Sensor Operators - Wait for External Conditions
from airflow import DAG
from airflow.sensors.filesystem import FileSensor
from airflow.sensors.external_task import ExternalTaskSensor
from airflow.providers.http.sensors.http import HttpSensor
from airflow.providers.amazon.aws.sensors.s3 import S3KeySensor
from airflow.operators.python import PythonOperator
from datetime import datetime, timedelta

with DAG(
    'sensor_operators_example',
    start_date=datetime(2024, 1, 1),
    schedule_interval='@daily',
    catchup=False,
) as dag:
    
    # Wait for file to appear
    wait_for_file = FileSensor(
        task_id='wait_for_file',
        filepath='/data/incoming/{{ ds }}/data.csv',
        poke_interval=60,  # Check every 60 seconds
        timeout=3600,      # Timeout after 1 hour
        mode='poke',       # 'poke' or 'reschedule'
        soft_fail=False,   # If True, marks as skipped on timeout
    )
    
    # Wait for S3 file
    wait_for_s3 = S3KeySensor(
        task_id='wait_for_s3_file',
        bucket_name='my-data-bucket',
        bucket_key='raw/{{ ds }}/events.parquet',
        aws_conn_id='aws_default',
        poke_interval=300,
        timeout=7200,
        mode='reschedule',  # Frees worker slot between checks
    )
    
    # Wait for API to be ready
    wait_for_api = HttpSensor(
        task_id='wait_for_api',
        http_conn_id='api_default',
        endpoint='/health',
        response_check=lambda response: response.status_code == 200,
        poke_interval=30,
        timeout=600,
    )
    
    # Wait for another DAG's task to complete
    wait_for_upstream_dag = ExternalTaskSensor(
        task_id='wait_for_upstream',
        external_dag_id='upstream_dag',
        external_task_id='final_task',
        execution_delta=timedelta(hours=0),  # Same execution date
        timeout=3600,
        mode='reschedule',
    )
    
    process = PythonOperator(
        task_id='process_data',
        python_callable=lambda: print("Processing..."),
    )
    
    # All sensors must complete before processing
    [wait_for_file, wait_for_s3, wait_for_api, wait_for_upstream_dag] >> process

### Sensor Modes

| Mode | Behavior | Worker Usage | Best For |
|------|----------|--------------|----------|
| `poke` | Holds worker slot | Continuous | Short waits, high-frequency checks |
| `reschedule` | Releases worker | Intermittent | Long waits, resource-constrained environments |

---

## 4. XCom for Task Communication <a id='xcom'></a>

**XCom (Cross-Communication)** allows tasks to exchange small amounts of data.

```
┌─────────────────────────────────────────────────────────────────────┐
│                           XCOM FLOW                                 │
├─────────────────────────────────────────────────────────────────────┤
│                                                                     │
│   ┌──────────┐         ┌──────────────┐         ┌──────────┐       │
│   │  Task A  │──push──►│   Metadata   │◄──pull──│  Task B  │       │
│   │          │         │   Database   │         │          │       │
│   └──────────┘         │   (XComs)    │         └──────────┘       │
│                        └──────────────┘                             │
│                                                                     │
│   Key: (dag_id, task_id, run_id, key)                              │
│   Value: Serialized data (JSON by default)                         │
│                                                                     │
└─────────────────────────────────────────────────────────────────────┘
```

### XCom Best Practices

| ✅ Do | ❌ Don't |
|-------|----------|
| Pass small metadata (IDs, paths, counts) | Pass large datasets |
| Use for file paths | Store entire DataFrames |
| Pass configuration values | Transfer binary files |
| Use custom XCom backends for large data | Exceed 48KB (default limit) |

In [ ]:
# XCom Examples - Multiple Methods
from airflow import DAG
from airflow.operators.python import PythonOperator
from airflow.decorators import task
from datetime import datetime

# Method 1: Traditional XCom with ti
def push_via_ti(ti):
    """Push to XCom using TaskInstance."""
    data = {'file_path': '/data/processed/output.parquet', 'row_count': 50000}
    ti.xcom_push(key='etl_metadata', value=data)
    ti.xcom_push(key='status', value='success')

def pull_via_ti(ti):
    """Pull from XCom using TaskInstance."""
    metadata = ti.xcom_pull(task_ids='push_data', key='etl_metadata')
    status = ti.xcom_pull(task_ids='push_data', key='status')
    print(f"File: {metadata['file_path']}, Rows: {metadata['row_count']}, Status: {status}")

# Method 2: Return value (auto-pushed with key='return_value')
def push_via_return():
    """Return value is automatically pushed to XCom."""
    return {'processed_at': datetime.now().isoformat(), 'records': 1000}

def pull_return_value(ti):
    """Pull the return value from another task."""
    result = ti.xcom_pull(task_ids='auto_push')  # No key needed for return_value
    print(f"Received: {result}")

# Method 3: Using Jinja templating
def process_with_template(**context):
    """Access XCom via templates in operator parameters."""
    print(f"Processing...")

with DAG(
    'xcom_examples',
    start_date=datetime(2024, 1, 1),
    schedule_interval='@daily',
    catchup=False,
) as dag:
    
    push_data = PythonOperator(
        task_id='push_data',
        python_callable=push_via_ti,
    )
    
    pull_data = PythonOperator(
        task_id='pull_data',
        python_callable=pull_via_ti,
    )
    
    auto_push = PythonOperator(
        task_id='auto_push',
        python_callable=push_via_return,
    )
    
    auto_pull = PythonOperator(
        task_id='auto_pull',
        python_callable=pull_return_value,
    )
    
    push_data >> pull_data
    auto_push >> auto_pull

In [ ]:
# Modern TaskFlow API with XCom (Airflow 2.0+)
from airflow.decorators import dag, task
from datetime import datetime

@dag(
    dag_id='taskflow_xcom_example',
    start_date=datetime(2024, 1, 1),
    schedule_interval='@daily',
    catchup=False,
    tags=['taskflow', 'modern'],
)
def taskflow_etl():
    """ETL pipeline using TaskFlow API with implicit XCom."""
    
    @task
    def extract() -> dict:
        """Extract data from source."""
        return {
            'data': [1, 2, 3, 4, 5],
            'source': 'api',
            'extracted_at': datetime.now().isoformat()
        }
    
    @task(multiple_outputs=True)  # Allows dict unpacking
    def transform(raw_data: dict) -> dict:
        """Transform extracted data."""
        transformed = [x * 2 for x in raw_data['data']]
        return {
            'transformed_data': transformed,
            'count': len(transformed),
            'sum': sum(transformed),
        }
    
    @task
    def load(transformed_data: list, count: int, sum_val: int) -> None:
        """Load data to destination."""
        print(f"Loading {count} records with sum {sum_val}")
        print(f"Data: {transformed_data}")
    
    @task
    def notify(message: str) -> None:
        """Send notification."""
        print(f"Notification: {message}")
    
    # XCom is handled implicitly!
    raw = extract()
    result = transform(raw)
    load(result['transformed_data'], result['count'], result['sum'])
    notify("ETL completed successfully!")

# Instantiate the DAG
taskflow_dag = taskflow_etl()

---

## 5. Best Practices for DAG Design <a id='best-practices'></a>

### ✅ DAG Design Principles

```
┌─────────────────────────────────────────────────────────────────────────┐
│                    DAG DESIGN BEST PRACTICES                            │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                         │
│  1. IDEMPOTENCY           │  Re-running produces same result            │
│  2. ATOMICITY             │  Tasks succeed or fail completely           │
│  3. INCREMENTAL LOADS     │  Process only new/changed data              │
│  4. SMALL TASKS           │  Single responsibility per task             │
│  5. AVOID TOP-LEVEL CODE  │  No heavy imports at module level           │
│                                                                         │
└─────────────────────────────────────────────────────────────────────────┘
```

### Configuration Management

| Approach | Use Case | Example |
|----------|----------|--------|
| **Variables** | Runtime config | API keys, thresholds |
| **Connections** | External systems | DB credentials, S3 |
| **Params** | DAG-level config | Passed at trigger time |
| **Environment Variables** | Sensitive data | AWS credentials |

### Common Anti-Patterns to Avoid

| ❌ Anti-Pattern | ✅ Better Approach |
|-----------------|--------------------|
| Heavy top-level imports | Import inside task functions |
| Hardcoded values | Use Variables/Connections |
| Large XCom payloads | Store in S3, pass path via XCom |
| Complex DAG logic | Use DAG factories or TaskGroups |
| Many sequential tasks | Parallelize where possible |
| No retries | Set appropriate retry policies |

In [ ]:
# Best Practices Example - Production-Ready DAG
from airflow import DAG
from airflow.operators.python import PythonOperator
from airflow.operators.empty import EmptyOperator
from airflow.utils.task_group import TaskGroup
from airflow.models import Variable
from datetime import datetime, timedelta
import logging

logger = logging.getLogger(__name__)

# Best Practice: Define default args separately
default_args = {
    'owner': 'data_engineering',
    'depends_on_past': False,
    'email_on_failure': True,
    'email_on_retry': False,
    'retries': 3,
    'retry_delay': timedelta(minutes=5),
    'retry_exponential_backoff': True,
    'max_retry_delay': timedelta(minutes=30),
    'execution_timeout': timedelta(hours=1),
}

# Best Practice: Use factory function for dynamic DAGs
def create_etl_dag(source_name: str, schedule: str):
    """Factory function to create ETL DAGs."""
    
    dag_id = f'etl_{source_name}'
    
    with DAG(
        dag_id=dag_id,
        default_args=default_args,
        description=f'ETL pipeline for {source_name}',
        schedule_interval=schedule,
        start_date=datetime(2024, 1, 1),
        catchup=False,
        tags=['etl', source_name, 'production'],
        doc_md=f"""## ETL Pipeline: {source_name}
        
        This DAG extracts, transforms, and loads data from {source_name}.
        
        **Owner:** Data Engineering
        **Schedule:** {schedule}
        """,
    ) as dag:
        
        start = EmptyOperator(task_id='start')
        
        # Best Practice: Use TaskGroups for organization
        with TaskGroup('extract_tasks', tooltip='Data extraction') as extract_group:
            
            def extract_data(source, **context):
                # Best Practice: Heavy imports inside function
                import pandas as pd
                from sqlalchemy import create_engine
                
                # Best Practice: Use Variables for config
                config = Variable.get(f'{source}_config', deserialize_json=True)
                logger.info(f"Extracting from {source} with config: {config}")
                
                # Best Practice: Return path, not data
                output_path = f"/data/raw/{source}/{context['ds']}/data.parquet"
                return output_path
            
            extract = PythonOperator(
                task_id='extract',
                python_callable=extract_data,
                op_kwargs={'source': source_name},
            )
        
        with TaskGroup('transform_tasks', tooltip='Data transformation') as transform_group:
            
            def transform_data(ti, **context):
                input_path = ti.xcom_pull(task_ids='extract_tasks.extract')
                logger.info(f"Transforming data from {input_path}")
                
                output_path = f"/data/processed/{source_name}/{context['ds']}/data.parquet"
                return output_path
            
            transform = PythonOperator(
                task_id='transform',
                python_callable=transform_data,
            )
        
        with TaskGroup('load_tasks', tooltip='Data loading') as load_group:
            
            def load_data(ti, **context):
                input_path = ti.xcom_pull(task_ids='transform_tasks.transform')
                logger.info(f"Loading data from {input_path}")
                return {'rows_loaded': 10000, 'status': 'success'}
            
            load = PythonOperator(
                task_id='load',
                python_callable=load_data,
            )
        
        end = EmptyOperator(
            task_id='end',
            trigger_rule='none_failed',  # Runs if no upstream failed
        )
        
        # Define dependencies
        start >> extract_group >> transform_group >> load_group >> end
    
    return dag

# Create DAGs dynamically
sources = [
    ('salesforce', '@daily'),
    ('hubspot', '@hourly'),
    ('stripe', '0 */4 * * *'),
]

for source_name, schedule in sources:
    globals()[f'dag_etl_{source_name}'] = create_etl_dag(source_name, schedule)

---

## 6. DAG Examples <a id='examples'></a>

### Complete Production Examples

In [ ]:
# Example 1: Data Lake Ingestion Pipeline
from airflow import DAG
from airflow.decorators import task, dag
from airflow.providers.amazon.aws.operators.s3 import S3CreateObjectOperator
from airflow.providers.amazon.aws.sensors.s3 import S3KeySensor
from airflow.operators.python import BranchPythonOperator
from airflow.operators.empty import EmptyOperator
from datetime import datetime, timedelta

@dag(
    dag_id='data_lake_ingestion',
    start_date=datetime(2024, 1, 1),
    schedule_interval='@hourly',
    catchup=False,
    default_args={
        'retries': 3,
        'retry_delay': timedelta(minutes=5),
    },
    tags=['data-lake', 'ingestion', 'production'],
)
def data_lake_ingestion():
    """Ingest data from multiple sources into data lake."""
    
    @task
    def check_source_availability() -> dict:
        """Check which data sources have new data."""
        # Simulate checking sources
        return {
            'api_source': True,
            'db_source': True,
            'file_source': False,
        }
    
    @task
    def ingest_api_data() -> str:
        """Ingest data from REST API."""
        import json
        import requests
        
        # Simulated API call
        data = {'records': 500, 'source': 'api'}
        output_path = 's3://data-lake/raw/api/{{ ds }}/data.json'
        return output_path
    
    @task
    def ingest_db_data() -> str:
        """Ingest data from database."""
        output_path = 's3://data-lake/raw/db/{{ ds }}/data.parquet'
        return output_path
    
    @task
    def merge_and_dedupe(paths: list) -> str:
        """Merge data from all sources and deduplicate."""
        output_path = 's3://data-lake/processed/{{ ds }}/merged.parquet'
        return output_path
    
    @task
    def update_catalog(path: str) -> None:
        """Update data catalog with new partition."""
        print(f"Registering {path} in catalog")
    
    # Pipeline logic
    sources = check_source_availability()
    api_path = ingest_api_data()
    db_path = ingest_db_data()
    merged_path = merge_and_dedupe([api_path, db_path])
    update_catalog(merged_path)

data_lake_dag = data_lake_ingestion()

In [ ]:
# Example 2: ML Pipeline with Branching Logic
from airflow import DAG
from airflow.decorators import task
from airflow.operators.python import BranchPythonOperator
from airflow.operators.empty import EmptyOperator
from datetime import datetime, timedelta

with DAG(
    dag_id='ml_training_pipeline',
    start_date=datetime(2024, 1, 1),
    schedule_interval='@weekly',
    catchup=False,
    tags=['ml', 'training'],
) as dag:
    
    @task
    def prepare_training_data():
        """Prepare and validate training dataset."""
        # Simulate data preparation
        return {
            'path': 's3://ml-data/training/dataset.parquet',
            'rows': 100000,
            'features': 50,
        }
    
    @task
    def train_model(data_info: dict):
        """Train the ML model."""
        return {
            'model_path': 's3://ml-models/model_v2.pkl',
            'accuracy': 0.95,
            'f1_score': 0.93,
        }
    
    @task
    def evaluate_model(model_info: dict) -> dict:
        """Evaluate model performance."""
        return {
            'accuracy': model_info['accuracy'],
            'meets_threshold': model_info['accuracy'] > 0.90,
        }
    
    def decide_deployment(ti):
        """Decide whether to deploy based on metrics."""
        metrics = ti.xcom_pull(task_ids='evaluate_model')
        if metrics['meets_threshold']:
            return 'deploy_model'
        return 'notify_failure'
    
    @task
    def deploy_model():
        """Deploy model to production."""
        print("Model deployed successfully!")
    
    @task
    def notify_failure():
        """Notify team of failed quality check."""
        print("Model did not meet quality threshold")
    
    # Pipeline definition
    data = prepare_training_data()
    model = train_model(data)
    metrics = evaluate_model(model)
    
    branch = BranchPythonOperator(
        task_id='decide_deployment',
        python_callable=decide_deployment,
    )
    
    deploy = deploy_model()
    notify = notify_failure()
    
    end = EmptyOperator(
        task_id='end',
        trigger_rule='none_failed_min_one_success',
    )
    
    metrics >> branch >> [deploy, notify] >> end

In [ ]:
# Example 3: Dynamic Task Mapping (Airflow 2.3+)
from airflow import DAG
from airflow.decorators import task
from datetime import datetime

with DAG(
    dag_id='dynamic_task_mapping',
    start_date=datetime(2024, 1, 1),
    schedule_interval='@daily',
    catchup=False,
    tags=['dynamic', 'parallel'],
) as dag:
    
    @task
    def get_partitions() -> list:
        """Get list of partitions to process."""
        # This could query a database or list S3 prefixes
        return [
            {'partition': 'us-east', 'size': 1000},
            {'partition': 'us-west', 'size': 800},
            {'partition': 'eu-west', 'size': 1200},
            {'partition': 'ap-south', 'size': 600},
        ]
    
    @task
    def process_partition(partition_info: dict) -> dict:
        """Process a single partition."""
        partition = partition_info['partition']
        size = partition_info['size']
        print(f"Processing partition {partition} with {size} records")
        return {
            'partition': partition,
            'processed': size,
            'status': 'success',
        }
    
    @task
    def aggregate_results(results: list) -> dict:
        """Aggregate results from all partitions."""
        total = sum(r['processed'] for r in results)
        return {
            'total_processed': total,
            'partitions': len(results),
            'all_successful': all(r['status'] == 'success' for r in results),
        }
    
    # Dynamic task mapping - creates N parallel tasks
    partitions = get_partitions()
    
    # .expand() creates one task instance per item in the list
    processed = process_partition.expand(partition_info=partitions)
    
    # Aggregate automatically waits for all mapped tasks
    final = aggregate_results(processed)

---

## 7. Takeaways <a id='takeaways'></a>

### 🎯 Key Concepts Summary

| Concept | Key Points |
|---------|------------|
| **Architecture** | Scheduler orchestrates, Workers execute, Metadata DB stores state |
| **DAGs** | Define workflows as code, set schedules, manage dependencies |
| **Operators** | Action (Python, Bash), Transfer (S3, DB), Sensor (wait for events) |
| **XCom** | Pass small data between tasks; use paths for large data |
| **TaskFlow API** | Modern Python syntax with implicit XCom handling |
| **Dynamic Mapping** | Create parallel tasks at runtime based on data |

### 📋 Production Checklist

```
✅ DAG Design
   □ Idempotent tasks (re-runnable without side effects)
   □ Proper retry configuration
   □ Appropriate timeouts set
   □ Meaningful task IDs and DAG documentation
   □ Use TaskGroups for organization

✅ Performance
   □ Avoid heavy top-level imports
   □ Use 'reschedule' mode for long-running sensors
   □ Parallelize where possible
   □ Set appropriate max_active_runs

✅ Data Handling
   □ Small metadata in XCom, large data in object storage
   □ Use Connections for credentials
   □ Use Variables for runtime configuration

✅ Monitoring
   □ Email alerts configured
   □ SLA monitoring enabled
   □ Meaningful logging in tasks
   □ Custom metrics where needed
```

### 📚 Additional Resources

- [Apache Airflow Documentation](https://airflow.apache.org/docs/)
- [Astronomer Guides](https://docs.astronomer.io/learn)
- [Airflow Best Practices](https://airflow.apache.org/docs/apache-airflow/stable/best-practices.html)
- [Provider Packages](https://airflow.apache.org/docs/apache-airflow-providers/index.html)